In [12]:
from ase import Atoms
from ase.build import molecule
import numpy as np
from scipy.spatial.transform import Rotation
from ase.visualize import view

def validate_positive_integer(value, name):
    """
    正の整数であることを検証する関数。
    """
    if not isinstance(value, int) or value <= 0:
        raise ValueError(f"{name} must be a positive integer.")

def validate_non_negative(value, name):
    """
    非負であることを検証する関数。
    """
    if value < 0:
        raise ValueError(f"{name} must be non-negative.")

def validate_atoms_object(obj, name):
    """
    ASEのAtomsオブジェクトであることを検証する関数。
    """
    if not isinstance(obj, Atoms):
        raise TypeError(f"{name} must be an ASE Atoms object.")

class RandomMoleculeStructure:
    def __init__(self, molecule_templates, num_molecules_per_type, cell_size, max_attempts=10000, overlap_threshold=2.0):
        self.molecule_templates = molecule_templates
        self.num_molecules_per_type = num_molecules_per_type
        self.cell_size = cell_size
        self.max_attempts = max_attempts
        self.overlap_threshold = overlap_threshold
        self.positions = np.empty((0, 3))
        
        # 入力の検証
        validate_positive_integer(max_attempts, "max_attempts")
        validate_non_negative(overlap_threshold, "overlap_threshold")

        for num, name in zip(num_molecules_per_type, ["num_molecules_per_type[{}]".format(i) for i in range(len(num_molecules_per_type))]):
            validate_positive_integer(num, name)

        if len(cell_size) != 3 or not all(isinstance(x, (int, float)) and x >= 0 for x in cell_size):
            raise ValueError("cell_size must be a list/tuple of three non-negative numbers.")
        
        

    def random_rotation_matrix(self):
        """
        ランダムな回転行列を生成する。
        """
        random_rotation = Rotation.from_rotvec(np.random.rand(3) * 2 * np.pi)
        return random_rotation.as_matrix()

    def check_overlap(self, new_pos):
        """
        分子同士の重なりをチェックし、新しい位置が重なる場合はTrueを返す。
        """
        if self.positions.size == 0:
            return False
        dists = np.linalg.norm(self.positions - new_pos, axis=1)
        return np.any(dists < self.overlap_threshold)

    def generate_structure(self):
        """
        分子をランダムな向きに回転させて配置する。一定回数以上の試行で適切な配置が見つからない場合はエラーを返す。
        """
        for mol_index, mol_template in enumerate(self.molecule_templates):
            num_molecules = self.num_molecules_per_type[mol_index]
            if num_molecules == 1:  # Place at the center
                self.positions = np.vstack([self.positions, np.array([self.cell_size[0] / 2, self.cell_size[1] / 2, 0.0])])
            else:
                attempts = 0
                while len(self.positions) < sum(self.num_molecules_per_type[:mol_index + 1]):
                    new_pos = np.random.rand(3) * self.cell_size
                    if not self.check_overlap(new_pos):
                        self.positions = np.vstack([self.positions, new_pos])
                    else:
                        attempts += 1
                        if attempts >= self.max_attempts:
                            raise RuntimeError("Failed to find non-overlapping configuration after {} attempts.".format(self.max_attempts))

        large_structure = Atoms(cell=self.cell_size)
        for i, pos in enumerate(self.positions):
            mol_index = 0
            while i >= sum(self.num_molecules_per_type[:mol_index + 1]):
                mol_index += 1
            mol_template = self.molecule_templates[mol_index]
            mol = mol_template.copy()
            self.rotation_matrix = self.random_rotation_matrix()
            mol.positions = np.dot(mol.positions, self.rotation_matrix.T)
            mol.translate(pos)
            large_structure += mol

        large_structure.set_pbc([True, True, True])
        return large_structure

# 使用例
num_molecules_per_type = [10, 10, 20,1]
cell_size = [10, 10, 10]
water_molecule = molecule('H2O')
carbon_dioxide_molecule = molecule('CO2')
methane_molecule = molecule('CH4')

molecule_templates = [water_molecule, carbon_dioxide_molecule, methane_molecule,molecule("CH3CH2NH2")]
random_structure = RandomMoleculeStructure(molecule_templates, num_molecules_per_type, cell_size)
structure = random_structure.generate_structure()
print(structure)
view(structure)

Atoms(symbols='C32H107NO30', pbc=True, cell=[10.0, 10.0, 10.0])


<Popen: returncode: None args: ['/home/kazuki/project/sample_chgnet/.venv/bi...>

In [20]:
from ase.build import (
    fcc100, fcc110, fcc111, fcc211, fcc111_root,
    bcc100, bcc110, bcc111, bcc111_root,
    hcp0001, hcp10m10, hcp0001_root,
    diamond100, diamond111
)

pt100_atoms = fcc100("Pt", size=(10, 10, 5), vacuum=2.0)
# view(pt100_atoms)
print(pt100_atoms.cell[0])

[27.71858582  0.          0.        ]


In [21]:
num_molecules_per_type = [200, 200, 200,1]
cell_size = [pt100_atoms.cell[0][0], pt100_atoms.cell[1][1], 30]
water_molecule = molecule('H2O')
carbon_dioxide_molecule = molecule('CO2')
methane_molecule = molecule('CH4')

molecule_templates = [water_molecule, carbon_dioxide_molecule, methane_molecule,molecule("CH3CH2NH2")]

random_structure = RandomMoleculeStructure(molecule_templates, num_molecules_per_type, cell_size)
structure = random_structure.generate_structure()
print(structure)
# view(structure)

Atoms(symbols='C402H1207NO600', pbc=True, cell=[27.71858582251266, 27.71858582251266, 30.0])


In [23]:
from ase.io import read, write
slab_cell = pt100_atoms.get_cell()
# structure.translate(slab_cell[0] / 2 + slab_cell[1] / 2+ slab_cell[2] / 2)
slab_center = np.mean(pt100_atoms.positions, axis=0)+structure.cell[2]/2+slab_cell[2] / 2+3
structure.translate(slab_center - np.mean(structure.positions, axis=0))
merged=pt100_atoms+structure
merged.set_cell((slab_cell[0], slab_cell[1], slab_cell[2]+structure.cell[2]+[0,0,40]))
merged.wrap()
view(merged)
print(len(merged))
write("sample.xyz", merged)

2710


/home/kazuki/project/sample_chgnet/.venv/lib/python3.12/site-packages/ase/io/extxyz.py:302: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


In [16]:
slab_center

array([12.70150504, 15.47336362, 22.        ])